# Global

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
#IPython.Cell.options_default.cm_config.lineNumbers = true;

################################################################################
## Python Version
################################################################################
import sys


################################################################################
## General Stuff
################################################################################
from multiprocessing import Pool
from tqdm import tqdm


################################################################################
## Util Stuff
################################################################################
from timeUtils import clock, elapsed
from ioUtils import saveFile, getFile


################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist


################################################################################
## Music Names
################################################################################
from masterArtistNameDB import masterArtistNameDB


################################################################################
## Chart Stuff
################################################################################
from artistIgnores import getArtistIgnores
from billboardData import billboardData
from top40Data import top40Data
from spotifyData import spotifyData
from chartArtistAlbumData import chartArtistAlbumData
from chartUtils import *
from extraArtists import extraKnownArtists


################################################################################
## Pandas Stuff
################################################################################
import pandas as pd
from pandas import DataFrame
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

chartType = None

print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2021-04-07 23:12:42.658750


# Charts To Merge

In [2]:
chartsToMerge = ["RYMList", "RYMList2"]

In [3]:
mdbmaps = {}
for chartType in chartsToMerge:
    mdbmaps[chartType] = musicDBMap(chartType)


Could not determine music db source type for db [RYMList]
  Valid DBs: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm', 'Deezer', 'AppleMusic', 'AlbumOfTheYear', 'Genius', 'IHeart', 'KWorbSpotify', 'KWorbiTunes', 'KWorbYouTube']
  Loaded 31177 previously matched entries
Could not determine music db source type for db [RYMList2]
  Valid DBs: ['Discogs', 'AllMusic', 'MusicBrainz', 'AceBootlegs', 'RateYourMusic', 'LastFM', 'DatPiff', 'RockCorner', 'CDandLP', 'MusicStack', 'MetalStorm', 'Deezer', 'AppleMusic', 'AlbumOfTheYear', 'Genius', 'IHeart', 'KWorbSpotify', 'KWorbiTunes', 'KWorbYouTube']
  Loaded 22236 previously matched entries


In [23]:
dbChartMap = {}
for chartType in chartsToMerge:
    for primaryKey, primaryData in mdbmaps[chartType].get().items():
        artistName = primaryData.artistName
        for db,dbData in primaryData.get().items():
            if dbChartMap.get(db) is None:
                dbChartMap[db] = {}
            dbID,name = dbData.get()
            if dbID is not None:
                if dbChartMap[db].get(dbID) is None:
                    dbChartMap[db][dbID] = {}
                if dbChartMap[db][dbID].get(artistName) is None:
                    dbChartMap[db][dbID][artistName] = {}
                dbChartMap[db][dbID][artistName][chartType] = primaryKey

In [26]:
DataFrame(dbChartMap["Discogs"])

,23755,59792,82730,70829,252864,138556,10263,90233,19812,224273,97545,200815,132136,215471,39766,196766,227848,41095,3840,36052,1373,102984,268383,202598,363443,592308,3268,94078,20991,144998,10343,18984,66358,34278,17589,45467,3865,18956,11879,28332,82294,4654,22854,634,29890,21764,85929,15118,28972,6520,219986,74500,83080,3909,7552,2725,17199,11136,36665,125246,388,10584,83609,10995,38561,76894,92577,137880,141549,38863,25261,10463,28795,39900,317007,1778977,261537,713862,1345479,365248,125433,985480,266385,2077774,243267,52835,17966,145257,145256,232157,71872,56798,48410,158676,21147,255411,255672,152680,152682,135946,2774,56147,50984,9130,29735,10824,3928,207958,281232,369298,172047,38948,955,275841,861083,1105843,445868,16533,3452157,64748,102994,252102,28085,70828,92623,370500,48432,17203,30700,35301,29923,40226,39768,22164,270582,56621,57103,106474,10082,138572,654544,5049,42058,252310,140646,71891,47333,21323,10384,55029,7987,15885,128581,192557,45115,83077,83242,12373,410281,2228,268769,60480,19731,253234,12368,10783,231577,28209,4480,68467,6516,7578,45,26494,50997,7551,3551,243300,3847,307,52833,27518,301147,295202,31260,8699,8760,21742,170355,50263,39776,160906,81013,48424,69866,255773,18839,129885,218269,37028,5105,2102143,306157,1247036,242615,3270231,200321,29979,2171152,1771611,3243777,1382634,1926684,269355,1536084,1042739,3268583,1059062,419677,349985,1581670,37749,3688452,346760,1475750,503060,1415073,3519986,3264326,64735,2077383,2013868,1759402,2788950,45642,33287,1245513,255386,229498,304455,252529,335809,254199,280442,3855,96922,3550,258372,116345,65791,...,1212192,1792495,1342040,1295449,1098233,137444,234333,1423736,1274294,4726261,2082052,226971,2012821,4537380,1372240,1014574,2504483,814051,813488,847531,1269886,1445533,292130,2644597,280709,1141309,3360569,1659438,515835,2664065,1915295,1224020,1342710,1295896,2673074,1350884,363198,1051345,424805,2763225,1277410,510576,548618,1357035,1240011,471026,950193,6448490,410643,322475,586915,2617220,941493,1103190,930201,2491244,453013,668744,884118,1766111,5220047,1086446,298078,2240247,1861414,1360467,1193640,648849,7155,44561,1198556,1202885,7486094,3401426,3525616,282385,1012712,2570059,170690,3077753,1316255,1678754,1545685,2535544,1036755,257606,4473510,1933157,517868,380651,2402419,3479119,894435,2628917,1463781,437878,5367727,2622116,4581174,4589969,528362,2374572,1132387,5196262,2157324,5185205,2938668,1975732,3574203,4371442,441499,349030,269599,1503056,1948740,2153961,2381388,2529792,1736735,91982,703636,3154497,1561757,362337,396568,1257936,1782831,428549,391237,985545,209403,5171222,391209,356104,320822,680992,1220578,254601,1951574,397169,282062,41818,423156,1227881,2216010,290316,1690564,1757928,379731,1230533,300019,2781571,2225099,372877,1279501,30137,1527090,382134,2858254,1856899,1157682,518132,1586666,1082136,1551837,292385,588226,164574,275118,529223,120708,254156,1796967,3080261,318593,356117,6465450,585309,136454,292446,928160,1127145,306727,4335641,1399647,4524975,4452158,4063092,714651,1775458,1638644,3962613,5646433,780251,7628827,177048,2991104,8057054,1109654,2118327,2380377,650293,3395193,868948,1551417,837755,2113841,3141914,640539,31219,1718845,25106,48077,1643578,1305755,75592,597233,204080,209555,5306567,5377468,339048,288131,256164,877487,309962,3085640,5718078,1525986,1943402,278541,520681,163793,1263100,364003,142347,553003,94939,97770,1227144,262766,1603880,1277031,652363,2496511,1668201,6606199,1062980,1049369,49262
Miles Davis,{'RYMList': 'e7f9fe3b71b3474b737a64c667d74530'...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [13]:
primaryData.get()

{'Discogs': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e86d0d10>,
 'AllMusic': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e9c1c510>,
 'MusicBrainz': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e9c1c590>,
 'AceBootlegs': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e9c1c5d0>,
 'RateYourMusic': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e9c1c610>,
 'LastFM': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e9c1c690>,
 'DatPiff': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e9c1c6d0>,
 'RockCorner': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e9c1c710>,
 'CDandLP': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e9c1c750>,
 'MusicStack': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e9c1c790>,
 'MetalStorm': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e9c1c7d0>,
 'Deezer': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e9c1c810>,
 'AppleMusic': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e9c1c850>,
 'AlbumOfTheYear': <musicDBArtistMap.myMusicDBIDData at 0x7fd1e9c1c890>,
 'Genius': <musicDBArtistMap.myMusicDBIDData